In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
device = "cuda"  if torch.cuda.is_available() else "cpu"
import numpy as np
import pandas as pd


In [3]:
X_train = np.load("/content/X_train_up.npy")
X_test = np.load("/content/X_test_up (1).npy")

In [ ]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components =13)

# X_train_tr = []

# for i in range(len(X_train)):

#     tt  =  pca.fit_transform(X_train[i])
#     X_train_tr.append(tt)
# X_train = np.array(X_train)

In [ ]:
# X_test_tr = []

# for i in range(len(X_test)):

#     tt  =  pca.fit_transform(X_test[i])
#     X_test_tr.append(tt)
# X_test = np.array(X_test)

In [4]:
X_train =  torch.tensor(X_train,requires_grad= True).to(device)
X_test =  torch.tensor(X_test,requires_grad = True).to(device)

In [26]:
X_train.shape

torch.Size([224, 750, 32])

In [5]:
y_train  = torch.load("/content/y_train.pt",map_location = device)
y_test = torch.load("/content/y_test.pt",map_location = device)


In [6]:
for i in range(len(y_train)):
    y_train[i]-=1;
for i in range(len(y_test)):
    y_test[i]-=1;


In [7]:
batch_size=10
train_loader = DataLoader(TensorDataset(X_train,y_train), batch_size=10, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test,y_test), batch_size=10,shuffle = True)


In [79]:
l1 = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=(3, 3), stride=1, padding=0).to(device)


l2 = nn.Conv2d(in_channels=5, out_channels=20, kernel_size=(3, 3), stride=1, padding=0).to(device)
l3 = nn.Conv2d(in_channels=20, out_channels=30, kernel_size=3, stride=1, padding=0,dtype = torch.float32)
pool = nn.MaxPool2d(kernel_size=(2, 2), stride=2).to(device)
print(X_train[0].shape)
print(pool(l3(pool(l2(pool(l1(X_train[0].unsqueeze(0))))))).shape)

torch.Size([750, 32])
torch.Size([30, 92, 2])


In [84]:
class CNNmodel(nn.Module):

    def __init__(self,num_classes):
        super(CNNmodel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, stride=1, padding=0,dtype = torch.float32)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3, stride=1, padding=0,dtype = torch.float32)
        self.conv3 = nn.Conv2d(in_channels=20, out_channels=30, kernel_size=3, stride=1, padding=0,dtype = torch.float32)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(30*92*2, 128,dtype = torch.float32)  # Adjust input size based on your data dimensions
        self.fc2 = nn.Linear(128, num_classes,dtype = torch.float32)

    def forward(self,x):

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 30*92*2)  # Adjust input size based on your data dimensions

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x



In [85]:
model = CNNmodel(9).to(device)
# model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [92]:
num_epochs = 5
for epoch in range(num_epochs):


    for batch_idx, (data, targets) in enumerate(train_loader):
        model.train()
        optimizer.zero_grad()
        data = data.to(device=device, dtype=torch.float32)
        outputs = model(data.unsqueeze(1))
        outputs = outputs.to(device =  device, dtype = torch.float32)
#         targets = targets.to(device=device, dtype=torch.float32)# Add a channel dimension
        # print(data.shape,targets.shape)
        loss = criterion(outputs, targets.squeeze())
        loss.backward()
        optimizer.step()
        if (batch_idx + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for data, targets in test_loader:
        outputs = model(data.unsqueeze(1))  # Add a channel dimension
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    print(f'Accuracy on test set: {(correct / total) * 100:.2f}%')

Epoch [1/5], Step [10/23], Loss: 0.0003
Epoch [1/5], Step [20/23], Loss: 0.0004
Epoch [2/5], Step [10/23], Loss: 0.0001
Epoch [2/5], Step [20/23], Loss: 0.0003
Epoch [3/5], Step [10/23], Loss: 0.0001
Epoch [3/5], Step [20/23], Loss: 0.0001
Epoch [4/5], Step [10/23], Loss: 0.0003
Epoch [4/5], Step [20/23], Loss: 0.0003
Epoch [5/5], Step [10/23], Loss: 0.0000
Epoch [5/5], Step [20/23], Loss: 0.0002
Accuracy on test set: 121.43%


In [93]:
train_acc = 0
with torch.inference_mode():
    for sequences,label in train_loader:
        sequences = sequences.to(device)
        label = label.to(device)
        preds = model(sequences.unsqueeze(1))

        preds =  torch.argmax(preds,axis =1)

        train_acc += (preds==label.squeeze()).sum()

print(f"accuracy on training set {train_acc/len(y_train)}")

accuracy on training set 1.0


In [94]:
test_acc = 0
with torch.inference_mode():
    for sequences,label in test_loader:
        sequences = sequences.to(device)
        label = label.to(device)
        preds = model(sequences.unsqueeze(1))

        preds =  torch.argmax(preds,axis =1)

        test_acc += (preds==label.squeeze()).sum()

print(f"accuracy on testing set {test_acc/len(y_test)}")

accuracy on testing set 0.4464285671710968


In [95]:
torch.save(model.state_dict(),"model_11_cnn.pth")

In [73]:
test_point = torch.load("/content/test_points.pt")

In [101]:
import pandas as pd
test = pd.read_excel("/content/Test.xlsx")

test = test.drop(columns= ["ID"])
test = test.values

print(test.shape)
test = torch.tensor(test,dtype = torch.float32)
def create_data_pt(random_tensor: torch.Tensor)->torch.Tensor:

    le =  int(random_tensor.shape[0]/32)

    arr = torch.zeros(le,random_tensor.shape[1],32).to(device)
    for i in range(le):

        data_pt = random_tensor[32*i: 32*(i+1),:]
        data_pt = data_pt.T
#         print(data_pt.shape)
        arr[i] =  data_pt


    return arr

test_points=  create_data_pt(test)

print(test_points.shape)

(8672, 750)
torch.Size([271, 750, 32])


In [103]:
torch.save(test_points,"og_test_points.pt")

In [104]:

preds = model(test_points.unsqueeze(1))
preds  = torch.argmax(preds ,axis=1)
preds = preds.numpy()



dictt = {0: 'Tiger',
    1: 'Snake',
    2: 'Wolf',
    3: 'Bear',
    4: 'Rabbit',
    5: 'Monkey',
    6: 'Eagle',
    7: 'Dolphin',
    8: 'Koala'}

arr1  = [ dictt[i]  for i in preds]
arr1

['Eagle',
 'Dolphin',
 'Rabbit',
 'Bear',
 'Koala',
 'Koala',
 'Rabbit',
 'Eagle',
 'Koala',
 'Bear',
 'Dolphin',
 'Snake',
 'Dolphin',
 'Monkey',
 'Rabbit',
 'Koala',
 'Bear',
 'Wolf',
 'Wolf',
 'Snake',
 'Rabbit',
 'Koala',
 'Snake',
 'Bear',
 'Snake',
 'Wolf',
 'Dolphin',
 'Wolf',
 'Snake',
 'Monkey',
 'Dolphin',
 'Snake',
 'Koala',
 'Eagle',
 'Rabbit',
 'Koala',
 'Rabbit',
 'Bear',
 'Bear',
 'Tiger',
 'Wolf',
 'Rabbit',
 'Wolf',
 'Koala',
 'Rabbit',
 'Tiger',
 'Snake',
 'Monkey',
 'Eagle',
 'Rabbit',
 'Eagle',
 'Bear',
 'Monkey',
 'Dolphin',
 'Rabbit',
 'Rabbit',
 'Bear',
 'Bear',
 'Dolphin',
 'Snake',
 'Monkey',
 'Rabbit',
 'Wolf',
 'Eagle',
 'Dolphin',
 'Dolphin',
 'Rabbit',
 'Tiger',
 'Wolf',
 'Dolphin',
 'Koala',
 'Monkey',
 'Wolf',
 'Koala',
 'Eagle',
 'Koala',
 'Rabbit',
 'Monkey',
 'Koala',
 'Koala',
 'Rabbit',
 'Dolphin',
 'Wolf',
 'Dolphin',
 'Tiger',
 'Tiger',
 'Koala',
 'Dolphin',
 'Monkey',
 'Koala',
 'Koala',
 'Monkey',
 'Snake',
 'Tiger',
 'Koala',
 'Koala',
 'Monkey'

In [97]:
test_point.shape

torch.Size([271, 750, 32])

In [75]:

# Create a tensor containing zero vectors with the same shape as the original tensor
zero_vector = torch.zeros(271, 1, 32)  # 1 is added to the second dimension to match the size [271, 750, 32]

# Concatenate the original tensor with the zero vector tensor along the second dimension
test_point = torch.cat((test_point, zero_vector), dim=1)

# Check the size of the new tensor
test_point.size()

torch.Size([271, 750, 32])

In [76]:
preds = model(test_point.unsqueeze(1))
preds  = torch.argmax(preds ,axis=1)
preds = preds.numpy()



dictt = {0: 'Tiger',
    1: 'Snake',
    2: 'Wolf',
    3: 'Bear',
    4: 'Rabbit',
    5: 'Monkey',
    6: 'Eagle',
    7: 'Dolphin',
    8: 'Koala'}

arr1  = [ dictt[i]  for i in preds]
arr1

['Snake',
 'Eagle',
 'Rabbit',
 'Bear',
 'Koala',
 'Eagle',
 'Wolf',
 'Snake',
 'Monkey',
 'Snake',
 'Snake',
 'Snake',
 'Dolphin',
 'Dolphin',
 'Wolf',
 'Snake',
 'Snake',
 'Koala',
 'Koala',
 'Snake',
 'Snake',
 'Tiger',
 'Snake',
 'Snake',
 'Koala',
 'Snake',
 'Eagle',
 'Dolphin',
 'Snake',
 'Snake',
 'Dolphin',
 'Snake',
 'Eagle',
 'Tiger',
 'Koala',
 'Dolphin',
 'Snake',
 'Rabbit',
 'Snake',
 'Tiger',
 'Snake',
 'Rabbit',
 'Wolf',
 'Wolf',
 'Dolphin',
 'Rabbit',
 'Snake',
 'Dolphin',
 'Snake',
 'Monkey',
 'Eagle',
 'Koala',
 'Snake',
 'Rabbit',
 'Koala',
 'Rabbit',
 'Snake',
 'Wolf',
 'Snake',
 'Wolf',
 'Bear',
 'Monkey',
 'Dolphin',
 'Eagle',
 'Snake',
 'Dolphin',
 'Rabbit',
 'Rabbit',
 'Wolf',
 'Dolphin',
 'Monkey',
 'Monkey',
 'Bear',
 'Rabbit',
 'Monkey',
 'Dolphin',
 'Snake',
 'Monkey',
 'Dolphin',
 'Rabbit',
 'Tiger',
 'Snake',
 'Monkey',
 'Dolphin',
 'Snake',
 'Koala',
 'Wolf',
 'Dolphin',
 'Monkey',
 'Wolf',
 'Snake',
 'Snake',
 'Snake',
 'Rabbit',
 'Dolphin',
 'Koala',
 '